In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
img_fl = np.load('../../downloading_data/images.npz')
ids = img_fl['ids']

In [4]:
def mag_to_flx(mag):
    """
    Input: an AB magnitude
    Output: a flux in microjansky (muJy)
    Works with np arrays and pd Series
    """
    return 10**(29-(48.60/2.5)) * 10**(-mag/2.5)

def read_lephare_spectrum_file(id):
    """
    Reads the file containing the output spectra for the inputted ID
    """
    with open(f'output_spectra/Id{str(id)[-9:]}.spec','r') as f:
        specfile_lines = f.read().split('\n')
    specfile_lines = [row.split() for row in specfile_lines]
    return specfile_lines

def get_lephare_spectrum(id, i): # i from 0 to 2
    """
    Inputs: An ID, and an integer in [0,1,2]
    Outputs: wavelengths, and a spectrum in muJy. The spectrum is the 
        LePHARE best fit spectrum for:
            0: Galaxy
            1: Quasar
            2: Star
    """
    file_lines = read_lephare_spectrum_file(id)
    spectra = np.array(file_lines[171:][:-1]).astype('float32')
    restart_indices = np.where(np.diff(spectra, axis=0)[:,0]<0)[0]
    spec_list = np.split(spectra, restart_indices+1, axis=0)
    x,y_mag = spec_list[i][:,0],spec_list[i][:,1]
    x,y_mag = x[(y_mag<100) & (y_mag!=0)], y_mag[(y_mag<100)&(y_mag!=0)] # Clipping ceilinged SED values
    y = mag_to_flx(y_mag)
    return x,y

Now the various wavelength and flux arrays generated by the above function are not of the same size (not just because some values have been clipped, it seems LePHARE always outputs non-standard lengths of spectra, so that's just great). In order to store the spectra in a nice way, and because we're only interested in a certain wavelength range anyway, we choose a standardised set of wavelength values between 0.4$\mu$m to 5$\mu$m and interpolate the LePHARE spectra.

In [5]:
num_samples = 1000
wavelength_samples = np.linspace(4e3, 5e4, num_samples)
spectrum_array = np.zeros((len(ids), 3, num_samples)) # ID x Spectrum Type (G/Q/S) x wavelength

for i, idd in tqdm(enumerate(ids), total=len(ids)):
    for obj_num in range(3):
        wavs, flxs = get_lephare_spectrum(idd,obj_num)
        flux_samples = np.interp(wavelength_samples,
                                 wavs, flxs)
        spectrum_array[i,obj_num,:] = flux_samples

100%|██████████████████████████████| 6690/6690 [03:51<00:00, 28.92it/s]


In [40]:
with open('lephare_output.out','r') as f:
    lephare_output_text = f.read()
lephare_df = pd.DataFrame([line.split() for line in lephare_output_text.split('\n')[55:]],
                          columns = ['COADD_OBJECT_ID',
                                     'Z_BEST',
                                     'Z_BEST68_LOW',
                                     'Z_BEST68_HIGH',
                                     'Z_ML',
                                     'CHI_BEST',
                                     'MOD_BEST',
                                     'MAG_ABS_BEST',
                                     'PDZ_BEST',
                                     'SCALE_BEST',
                                     'DIST_MOD_BEST',
                                     'NBAND_USED',
                                     'Z_SEC',
                                     'CHI_SEC',
                                     'MOD_SEC',
                                     'AGE_SEC',
                                     'Z_QSO',
                                     'CHI_QSO',
                                     'MOD_QSO',
                                     'MAG_ABS_QSO',
                                     'DIST_MOD_QSO',
                                     'MOD_STAR',
                                     'CHI_STAR',
                                     'CONTEXT',
                                     'ZSPEC'
                                    ]
                         )[:-1].set_index('COADD_OBJECT_ID')
lephare_df

,Z_BEST,Z_BEST68_LOW,Z_BEST68_HIGH,Z_ML,CHI_BEST,MOD_BEST,MAG_ABS_BEST,PDZ_BEST,SCALE_BEST,DIST_MOD_BEST,...,AGE_SEC,Z_QSO,CHI_QSO,MOD_QSO,MAG_ABS_QSO,DIST_MOD_QSO,MOD_STAR,CHI_STAR,CONTEXT,ZSPEC
COADD_OBJECT_ID,,,,,,,,,,,,,,,,,,,,,
931684428,0.0000,0.0000,0.0000,-99.0000,0.238356E+07,6,-13.703,100.000,0.323892E-01,0.000000E+00,...,-.990000E+02,0.3600,0.108625E+07,5,-25.167,0.414170E+02,53,0.368876E+07,511.0000,-.990000E+02
556041733,0.0000,0.0000,0.0000,-99.0000,0.493507E+07,6,-13.375,100.000,0.196960E-01,0.000000E+00,...,-.990000E+02,0.3600,0.340810E+07,21,-24.504,0.414170E+02,249,0.556088E+07,511.0000,-.990000E+02
979002481,0.0000,0.0000,0.0000,-99.0000,0.468933E+07,6,-13.589,100.000,0.239905E-01,0.000000E+00,...,-.990000E+02,0.4000,0.321413E+07,21,-25.014,0.416860E+02,18,0.361509E+07,511.0000,-.990000E+02
995403105,0.3662,0.3600,0.3600,-99.0000,0.494479E+07,6,-25.290,100.000,0.107130E+16,0.414603E+02,...,-.990000E+02,1.2800,0.467912E+07,28,-29.573,0.447649E+02,65,0.648082E+07,511.0000,-.990000E+02
979097465,0.3580,0.3600,0.3600,-99.0000,0.249094E+07,6,-25.098,100.000,0.700396E+15,0.414026E+02,...,-.990000E+02,0.0000,0.249717E+07,16,-13.238,0.000000E+00,51,0.342737E+07,511.0000,-.990000E+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887791189,0.8460,0.8379,0.8440,0.8403,0.417359E+03,1,-22.922,100.000,0.177745E+06,0.436535E+02,...,-.990000E+02,5.8800,0.872413E+02,21,-26.253,0.487571E+02,183,0.468758E+02,510.0000,-.990000E+02
697986932,0.7861,0.7961,0.8007,0.7998,0.725582E+03,1,-22.712,100.000,0.137934E+06,0.434572E+02,...,-.990000E+02,5.8800,0.687391E+02,14,-26.071,0.487571E+02,190,0.391549E+02,510.0000,-.990000E+02
11682001,1.0735,1.0749,1.0826,1.0793,0.110363E+03,1,-24.333,100.000,0.631149E+06,0.442923E+02,...,-.990000E+02,1.4000,0.987429E+02,10,-24.980,0.450055E+02,178,0.433225E+03,511.0000,-.990000E+02


The above df has 7102 objects, some of which don't have images (there are 6690 objects with 5-band imaging, in `ids`)

In [45]:
lephare_df_with_imgs = pd.DataFrame(lephare_df.loc[[str(int(str(idd)[-9:])) for idd in ids]].to_numpy(),
                                    columns = lephare_df.columns,
                                    index=ids)
lephare_df_with_imgs.to_csv('lephare_output_data.csv')
lephare_df_with_imgs

,Z_BEST,Z_BEST68_LOW,Z_BEST68_HIGH,Z_ML,CHI_BEST,MOD_BEST,MAG_ABS_BEST,PDZ_BEST,SCALE_BEST,DIST_MOD_BEST,...,AGE_SEC,Z_QSO,CHI_QSO,MOD_QSO,MAG_ABS_QSO,DIST_MOD_QSO,MOD_STAR,CHI_STAR,CONTEXT,ZSPEC
870313315,0.7002,0.6433,0.7270,0.6896,0.208101E+03,1,-22.531,100.000,0.113825E+06,0.431492E+02,...,-.990000E+02,5.8400,0.374989E+02,14,-26.293,0.487399E+02,193,0.359675E+02,510.0000,-.990000E+02
870331581,0.7398,0.7129,0.7679,0.7404,0.186506E+03,1,-22.795,100.000,0.166056E+06,0.432955E+02,...,-.990000E+02,5.5200,0.604831E+02,24,-27.155,0.485972E+02,167,0.413910E+02,510.0000,-.990000E+02
870480016,0.6580,0.6375,0.6805,0.6558,0.346023E+03,1,-22.554,100.000,0.133249E+06,0.429843E+02,...,-.990000E+02,5.8800,0.153644E+03,14,-26.930,0.487571E+02,193,0.112994E+03,511.0000,-.990000E+02
870527456,0.9592,0.9554,0.9642,0.9598,0.236991E+03,1,-23.807,100.000,0.393762E+06,0.439899E+02,...,-.990000E+02,5.9200,0.149271E+03,17,-27.923,0.487743E+02,182,0.879206E+02,510.0000,-.990000E+02
870548042,0.8272,0.8365,0.8408,0.8399,0.838074E+03,1,-23.741,100.000,0.357790E+06,0.435934E+02,...,-.990000E+02,5.8800,0.122053E+03,21,-27.153,0.487571E+02,184,0.116099E+03,510.0000,-.990000E+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700117776,1.2003,1.1934,1.2068,1.2002,0.885571E+02,2,-24.159,100.000,0.249152E+06,0.445922E+02,...,-.990000E+02,1.0400,0.113553E+03,2,-23.849,0.442070E+02,169,0.642548E+03,511.0000,-.990000E+02
1700212535,1.2186,1.1985,1.2401,1.2154,0.263384E+03,2,-24.588,100.000,0.384098E+06,0.446329E+02,...,-.990000E+02,1.2000,0.329767E+03,10,-24.673,0.445915E+02,169,0.139564E+04,511.0000,-.990000E+02
1700230002,0.8707,0.8784,0.8806,-99.0000,0.228409E+04,1,-24.066,100.000,0.488266E+06,0.437305E+02,...,-.990000E+02,5.6000,0.411480E+03,24,-27.666,0.486336E+02,181,0.194656E+03,511.0000,-.990000E+02
1700237805,1.2558,1.2368,1.2672,1.2529,0.255522E+03,2,-24.522,100.000,0.358862E+06,0.447136E+02,...,-.990000E+02,1.2400,0.201496E+03,10,-24.622,0.446796E+02,169,0.108570E+04,511.0000,-.990000E+02
